In [ ]:
%load_ext nb_black

In [1]:
### Magic functions
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

## Generating Data

In [1]:
#import 

seedNum = 888
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow.keras.datasets import mnist
import numpy as np
np.random.seed(seedNum)
import random
import matplotlib.pyplot as plt

from tensorflow import keras
from datetime import datetime
import time
from tqdm import tqdm

import torch

#import autokeras as ak
#import keras_tuner as kt


#load data


In [2]:
def get_function(func_str):
    if func_str == "abs":
        return (lambda x: np.abs(x).flatten(), 
                lambda x: torch.abs(x).flatten())
    elif func_str == "log":
        return (lambda x: np.log(np.abs(x)).flatten(), 
                lambda x: torch.log(torch.abs(x)).flatten())
    elif func_str == "sin":
        return (lambda x: np.sin(x).flatten(), 
                lambda x: torch.sin(x).flatten())
    else:
        return (lambda x: np.sign(np.abs(np.abs(x)-5)-2).flatten(), 
                lambda x: torch.sign(torch.abs(torch.abs(x)-5)-2).flatten())

In [3]:
def Simdata(NUM_I,seed,func,rho): #a = y_train/test, b = X_train/test
    
    np.random.seed(seed)    
    X_1_a_j = [] 
    X_2_a_j = [] 
    X_1_a_k = [] 
    X_2_a_k = [] 
    X_1_b_j = [] 
    X_2_b_j = [] 
    X_1_b_k = [] 
    X_2_b_k = []
    Z       = []
    for i in tqdm(range(0,NUM_I)):
        J = np.random.randint(4,10) # number of choice
        X_1_a = np.random.uniform(-1,1,J) #customer a
        X_2_a = np.random.uniform(-10,10,J)  #customer a
        X_1_b = np.random.uniform(-1,1,J) #customer b
        X_2_b = np.random.uniform(-10,10,J) #customer b
        
        xi  = np.random.normal(0,0.5,J)    # same across all customers
        
        X_2_a = X_2_a + rho*xi  #customer a endogeneity
        
        X_2_b = X_2_b + rho*xi  #customer b endogeneity
        
        u_a   = X_1_a + 2*func(X_2_a) + xi + np.random.normal(0,0.1,J) # \epsilon_{a}
        u_b   = X_1_b + 2*func(X_2_b) + xi + np.random.normal(0,0.1,J) # \epsilon_{b}
        choice_j = np.argmax(u_a) # return the index of product in the sample that customer a chose, we assume customer a as choose j

        choice_k = np.argmax(u_b) # return the index of product in the sample that customer b chose, we assume customer b as choose k
               
        if choice_j == choice_k:
            continue
        else:  

            X_1_a_j.append(X_1_a[choice_j])
            X_2_a_j.append(X_2_a[choice_j])
            X_1_a_k.append(X_1_a[choice_k])
            X_2_a_k.append(X_2_a[choice_k])
            
            X_1_b_j.append(X_1_b[choice_j]) 
            X_2_b_j.append(X_2_b[choice_j]) 
            X_1_b_k.append(X_1_b[choice_k]) 
            X_2_b_k.append(X_2_b[choice_k]) 
            Z.append(np.array([X_1_a[choice_j], X_2_a[choice_j], X_1_a[choice_k], X_2_a[choice_k], 
                                   X_1_b[choice_j], X_2_b[choice_j], X_1_b[choice_k], X_2_b[choice_k]]))
    
    return torch.Tensor(X_1_a_j).reshape((-1,1)).double(), torch.Tensor(X_2_a_j).reshape((-1,1)).double(), \
torch.Tensor(X_1_a_k).reshape((-1,1)).double(), torch.Tensor(X_2_a_k).reshape((-1,1)).double(), torch.Tensor(X_1_b_j).reshape((-1,1)).double(), \
torch.Tensor(X_2_b_j).reshape((-1,1)).double(), torch.Tensor(X_1_b_k).reshape((-1,1)).double(), torch.Tensor(X_2_b_k).reshape((-1,1)).double(), \
torch.tensor(Z, dtype=torch.float64) 

In [4]:
aa,bb = get_function("abs")

In [5]:
X_1_a_j, X_2_a_j, X_1_a_k, X_2_a_k, X_1_b_j, X_2_b_j, X_1_b_k, X_2_b_k, Z = Simdata(10000,2,aa,0.2)


100%|██████████| 10000/10000 [00:00<00:00, 33256.06it/s]
/tmp/ipykernel_174/1481296432.py:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  torch.tensor(Z, dtype=torch.float64)


## Training

In [6]:
X_1_a_j.shape

torch.Size([8286, 1])

In [7]:
Z.double()

tensor([[ 0.8631, -6.7995, -0.0305,  ...,  1.8479,  0.3661, -4.9309],
        [ 0.0105, -6.6935, -0.5965,  ..., -1.2046,  0.4015, -3.1482],
        [-0.5745, -9.4746, -0.0725,  ..., -2.1906, -0.5614, -9.4816],
        ...,
        [ 0.5541,  9.0974,  0.0371,  ..., -0.9986, -0.4890,  9.5518],
        [ 0.3592,  8.9569,  0.8240,  ..., -3.2084,  0.6406, -7.8417],
        [ 0.3701, -9.1316, -0.5504,  ..., -0.5643,  0.2180, -9.4245]],
       dtype=torch.float64)

In [8]:
import sys
import os
sys.path
print(sys.path)
sys.path.append('/notebooks/AdversarialGMM/')

['/notebooks/AdversarialGMM/local_notebooks', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']


In [9]:
import pickle
with open('sim_deepci_singlevar.pkl', 'rb') as f:
    pa, pb, ea, eb, ca_type, cb_type, g_ca_pa, g_ca_pb, g_cb_pa, g_cb_pb = pickle.load(f)
    
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [10]:
### module imports
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import itertools
import os

### import from our files
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import log_metrics, plot_results, hyperparam_grid,\
                                     hyperparam_mult_grid, eval_performance
from mliv.neuralnet.mnist_dgps import AbstractMNISTxz
from mliv.neuralnet import AGMM,KernelLayerMMDGMM
from mliv.neuralnet.rbflayer import gaussian, inverse_multiquadric

In [11]:
device = torch.cuda.current_device() if torch.cuda.is_available() else None

In [12]:
device

0

In [13]:
class CNN_Z_agmm(nn.Module):
    def __init__(self):
        super(CNN_Z_agmm, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = x  # F.log_softmax(x, dim=1)
        return output.squeeze()


class CNN_Z_kernel(nn.Module):
    def __init__(self, g_features=100):
        super(CNN_Z_kernel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, g_features)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = x  # F.log_softmax(x, dim=1)
        return output.squeeze()


class CNN_X(nn.Module):
    def __init__(self):
        super(CNN_X, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.view(x.shape[0], 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = x  # F.log_softmax(x, dim=1)
        return output.squeeze()

In [14]:
def fc_z_kernel(n_z, n_hidden, g_features, dropout_p):
    FC_Z_kernel = nn.Sequential(
        nn.Dropout(p=dropout_p),
        nn.Linear(n_z, n_hidden),
        nn.LeakyReLU(),
        nn.Dropout(p=dropout_p),
        nn.Linear(n_hidden, g_features),
        nn.ReLU(),
    )
    return FC_Z_kernel


def fc_z_agmm(n_z, n_hidden, dropout_p):
    FC_Z_agmm = nn.Sequential(
        nn.Dropout(p=dropout_p),
        nn.Linear(n_z, n_hidden),
        nn.LeakyReLU(),
        nn.Dropout(p=dropout_p),
        nn.Linear(n_hidden, 1),
    )
    return FC_Z_agmm


def fc_x(n_t, n_hidden, dropout_p):
    FC_X = nn.Sequential(
        nn.Dropout(p=dropout_p),
        nn.Linear(n_t, n_hidden),
        nn.LeakyReLU(),
        nn.Dropout(p=dropout_p),
        nn.Linear(n_hidden, 1),
    )
    return FC_X

In [15]:
k = 500
n_hidden = 100
net_learner = torch.nn.Sequential(
            torch.nn.Linear(1, k),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(k, 200),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(200, 1),
            )

#learner = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_hidden), nn.LeakyReLU(),
#                        nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ReLU(),
#                        nn.Dropout(p=p), nn.Linear(n_hidden, 1))

#adversary_fn = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_z, n_hidden), nn.LeakyReLU(),
#                             nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ReLU(),
#                             nn.Dropout(p=p), nn.Linear(n_hidden, 1))

net_adversary = torch.nn.Sequential(nn.Dropout(p=0.1),
            torch.nn.Linear(8, k),
            torch.nn.LeakyReLU(),
            nn.Dropout(p=0.1),                        
            torch.nn.Linear(k, 200),
            torch.nn.LeakyReLU(),
            nn.Dropout(p=0.1),                        
            torch.nn.Linear(200, 1),
            )
 
    

In [16]:
learner = net_learner

n_instruments = 1
n_hidden = 100
dropout_p = 0.1
adversary = net_adversary #fc_z_agmm(n_instruments, n_hidden, dropout_p)

In [17]:
from mliv.neuralnet import ADeepCI

In [18]:
res = ADeepCI(learner, adversary).fit(X_1_a_j, X_2_a_j, X_1_a_k, X_2_a_k, 
                  X_1_b_j, X_2_b_j, X_1_b_k, X_2_b_k, Z, 
            learner_l2=1e-3, adversary_l2=1e-4, adversary_norm_reg=1e-3,
            learner_lr=0.001, adversary_lr=0.001, n_epochs=2, bs=100, train_learner_every=3, train_adversary_every=1,
            ols_weight=0., warm_start=False, logger=None, model_dir='.', device=None, verbose=1)

Epoch #0


RuntimeError: expected scalar type Float but found Double

In [ ]:
model_final = torch.load(os.path.join(res.model_dir,"epoch{}".format(res.n_epochs - 1)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(X_2_a_j.T.squeeze().cpu().data.numpy(), model_final(X_2_a_j).cpu().data.numpy())
plt.show()

In [19]:
np.log(np.abs(pb.cpu().data.numpy())).flatten()

array([0.6003134 , 2.0740824 , 0.09482794, ..., 2.2298036 , 0.55139816,
       2.16771   ], dtype=float32)